In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import analyze # takes some time since inits hax
from channel_dict import channel_dict
from tqdm import tqdm


%matplotlib inline

Using TensorFlow backend.


In [2]:
#skipping runlist and submission stuff, should be done at this point

ACTUALLY_OFF = [1, 2, 12, 26, 34, 62, 65, 79, 86, 88, 102, 118, 
                130, 134, 135, 139, 148, 150, 152, 162, 178, 183,
                190, 198, 206, 213, 214, 234, 239, 244]

In [3]:
#takes runlists and uses just bottom runs 
#finds acceptances and errors

def data_exists(run_number):
    return os.path.exists('./data/run_%05d.h5' % run_number)

def all_data_exists(runlist):
    return all([data_exists(run) for run in runlist])

def file_to_list(runlist_file):
    return [int(run) for run in runlist_file.split('.')[0].split('_')[1:4]]

runlists = [f for f in os.listdir('./runlists')]

bottom_runs = []
accs = []
errors = []

missing_runs = []

for f in tqdm(sorted(runlists)):
    runlist = file_to_list(f)
    if not all_data_exists(runlist):
        #print('data missing for %s' % f)
        for r in runlist:
            if not data_exists(r):
                missing_runs.append(r)
        continue
    bottom_run = runlist[0]
    topbulk_run = runlist[1]
    topring_run = runlist[2]
    
    thresholds = analyze.get_thresholds(analyze.find_regular_run(bottom_run))
    
    acc, acc_errs = analyze.acceptance_3runs(bottom_run, topbulk_run, topring_run, thresholds)
    occ, occ_errs = analyze.occupancy_3runs(bottom_run, topbulk_run, topring_run)
    
    on_channels = np.where(occ > 0.05)[0]
    
    if len(on_channels) < 200:
        print('LED likely OFF for %s' % f)
        continue
    
    acc = acc[on_channels]
    acc_errs = acc_errs[on_channels]
    
    accs.append(np.median(acc))
    errors.append(np.sqrt((acc_errs**2).sum()) / len(on_channels))
    bottom_runs.append(bottom_run)
    

 18%|█▊        | 18/100 [00:39<03:04,  2.25s/it]/home/awalsh272/SPE/SPE/analyze.py:63: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - residual.cumsum(axis=1) / residual.sum(axis=1)[:, np.newaxis]
/home/awalsh272/SPE/SPE/analyze.py:63: RuntimeWarning: invalid value encountered in true_divide
  return 1 - residual.cumsum(axis=1) / residual.sum(axis=1)[:, np.newaxis]
/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/numpy/core/_methods.py:112: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
 29%|██▉       | 29/100 [01:10<03:06,  2.63s/it]

LED likely OFF for runlist_13433_13434_13435.txt


 32%|███▏      | 32/100 [01:18<02:58,  2.63s/it]

LED likely OFF for runlist_14088_14089_14090.txt


 54%|█████▍    | 54/100 [01:48<01:47,  2.35s/it]/home/awalsh272/SPE/SPE/analyze.py:31: RuntimeWarning: divide by zero encountered in log
  occupancy_array[:,i] = -1*np.log(self.make_correction(val, 'amplitude'))
100%|██████████| 100/100 [03:29<00:00,  2.59s/it]


In [4]:
#check individual channel and run number using plot_channel

ch='153' #user input
runnum='14089' #user input

#if you want to save the plots, add a path to filedir
analyze.plot_channel(ch, runnum, xlims=(-100,500), ylims=(-100,500), filedir='')

In [ ]:
#prints individual acceptance plots
plots_path='/project/lgrandi/xenon1t/spe_acceptance/plots'

run_number='14089' #user input, checks if plot already exists
if os.path.exists(plots_path):
    continue
else:
    analyze.plot_acceptances(accs, plots_path)

In [ ]:
means = []
medians = []
stds = []
bot_runs = []

n_bins = 40

acc_array = np.ones((len(runlist_files), len(channel_dict['all_channels'])))


for file, runlist in enumerate(sorted(runlist_files)):
    runlist = runlist.split('/')[-1].split('_')
    bottom_run = int(runlist[1])
    topbulk_run = int(runlist[2])
    topring_run = int(runlist[3][:-4])
    csv_file = "./acceptance_data/acceptances_%d_%d_%d.csv" % (bottom_run, topbulk_run, topring_run)
    
    if os.path.exists(csv_file):
        continue
        
        
    existing_files = [os.path.exists('/home/ershockley/analysis/SPE/data/run_%d' % r)
                        for r in [bottom_run, topbulk_run, topring_run]]
    if not all([os.path.exists('/home/ershockley/analysis/SPE/data/run_%d' % r)
                for r in [bottom_run, topbulk_run, topring_run]]):
        missing_runs = [str(run) for run, boo in zip([bottom_run, topbulk_run, topring_run],
                                               existing_files) if not boo]
        print("Missing data for runs %s" % ",".join(missing_runs))
        continue
    
    threshold_run = find_regular_run(bottom_run)
    print("Threshold run: %d" % threshold_run)
    try:
        thresholds = analyze.get_thresholds(threshold_run)
    except KeyError:
        thresholds = analyze.get_thresholds(threshold_run + 1)
        
    acceptances = analyze.get_acceptances_3runs(bottom_run, topring_run, 
                                                topbulk_run, thresholds, plot=True)
    
    
    acc_array[file, :] *= acceptances
    

    with open(csv_file, "w") as f:
        f.write("channel,acceptance\n")
        for ch, acc in enumerate(acceptances):
            f.write("%d,%0.4f\n" % (ch, acc))
        
    on_accs = acceptances
    on_accs = np.delete(on_accs, analyze.excluded_pmts)
    
    means.append(np.mean(on_accs))
    medians.append(np.median(on_accs))
    stds.append(np.std(on_accs))
    bot_runs.append(bottom_run)
    plt.figure()
    acc_hist, bins, patches = plt.hist(acceptances, bins = n_bins, range = (0,1))
    plt.title("Runs %d %d %d" % (bottom_run, topbulk_run, topring_run))
    plt.xlabel("SPE acceptance fraction")
    plt.ylabel(" # channels / bin (%d bins)" % n_bins)
    plt.show()
    #plt.savefig("/project/lgrandi/xenon1t/spe_acceptance/plots/hist_%d-%d-%d.png" % 
    #        (bottom_run, topbulk_run, topring_run))
    
print('DONE')
